In [3]:
# prob won't need all these but this is just my copy paste
import pandas as pd
import numpy as np
import csv
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, BertConfig,AdamW, BertForSequenceClassification,get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import accuracy_score, matthews_corrcoef, f1_score

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F

from tqdm import tqdm, trange, tnrange
import random
import os
import io
import nltk

In [ ]:
# load the model and test it
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=5,
                                                      output_attentions=False,
                                                      output_hidden_states=False)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

print(device)

model.load_state_dict(torch.load('finetuned_BERT_FairyGarbHVCHECK_epoch_3.model', map_location=torch.device('cuda')))

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

MAX_LENGTH = 256

In [ ]:
def encode(text_arr_in):
    encoded_data = tokenizer.batch_encode_plus(
    text_arr_in, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=MAX_LENGTH, 
    truncation = True,
    return_tensors='pt'
    )
    
    return encoded_data

In [ ]:
def getIDs(encoded_data_in):
    input_ids_train = encoded_data_in['input_ids']
    attention_masks_train = encoded_data_in['attention_mask']
    
    return input_ids_train, attention_masks_train

In [ ]:
def preproc_bert(text_in):
  encodings = encode(text_in)
  inputIDS, attention_masks = getIDs(encodings)
  return inputIDS, attention_masks

# encodings = encode(sentences)

# inputIDs, attention_masks = getIDs(encodings)
# print(inputIDs[0])
# print(attention_masks[0])

In [ ]:
mangid, mangmask = preproc_bert(mangost_sent)

In [ ]:
# l is the inputIDs and attention masks you are feeding in
# n is the number of chunks you want to output (I use 25)
def divide_chunks(l, n):
  # looping till length l
  for i in range(0, len(l), n): 
      yield l[i:i + n]

In [ ]:
n = 32
mango_inchunk = list(divide_chunks(mangid, n))

In [ ]:
def evaluate_text(input_chunk_in, att_chunk_in):
  pred_arr = []

  for i in range(len(input_chunk_in)):
    input = input_chunk_in[i]
    att = att_chunk_in[i]
    input = input.to(device)
    att = att.to(device)
    output = model(input, att, return_dict=False)
    _,prediction = torch.max(output[0], dim = 1)
    pred_arr.append(prediction.tolist())

  return pred_arr